In [1]:
import os
import pandas
import numpy as np
import nibabel as ni
from glob import glob

In [ ]:
def w_score_transformation(image_dir, spreadsheet, beta_dir, outdir, 
                           intercept='first', res_str = 'Res_', beta_str = 'beta',
                          cols_to_use = [], memory_load = 'high', img_mask = None):
    
    # check inputs
    print('initating and checking inputs')
    for cdir in [image_dir, beta_dir]:
        if type(cdir) != str:
            raise TypeError('%s must be a path pointing to a valid directory, you passed a %s object'%(cdir,
                                                                                                    type(cdir)))
        if not os.path.isdir(image_dir):
            raise IOError('could not find the directory specified in argument "image_dir"')
    if type(outdir) != str:
        raise TypeError('%s must be a path pointing to a valid directory, you passed a %s object'%(outdir,
                                                                                                type(cdir)))
    if not os.path.isdir(outdir):
        os.mkdir(outdir)
    
    if intercept not in ['first','last', 'none']:
        raise IOError('intercept must be set to "first", "last" or "none". See docstring for more info')
    if memory_load not in ['high','low']:
        raise IOError('memory_load must be set to "high" or "low". See docstring for more info')
    
    # load inputs
    raw_paths = glob(os.path.join(image_dir,'*'))
    res_paths = glob(os.path.join(beta_dir,'%s*'%res_str))
    if len(raw_paths) != len(res_paths):
        raise IOError('Inconsistency between raw (n=%s) and residual (n=%s) images, please revise'%(len(raw_paths),
                                                                                                   len(res_paths)))
    else:
        print('found %s images to transform'%len(raw_paths))
    beta_paths = glob(os.path.join(beta_dir,'%s*'%beta_str))
    if len(beta_paths) == 0:
        raise IOError('No beta images found in specified directory. Please revise beta_dir or beta_str arguments')
    if intercept == 'none':
        n_betas = len(beta_paths)
    else:
        n_betas = len(beta_paths) - 1
    if len(cols_to_use) > 0:
        if len(cols_to_use) != n_betas:
            raise IOError('# of columns in cols_to_use (%s) must match number of non-intercept beta maps (%s)'%(
                                                                                            len(cols_to_use),
                                                                                            n_betas))
    print('preparing spreadsheet')
    if type(spreadsheet) = str
        if '.csv' in spreadsheet:
            df = pandas.read_csv(spreadsheet)
        elif 'xl' in spreadsheet[-4:]:
            try:
                jnk = pandas.ExcelFile(spreadsheet)
                df = pandas.ExcelFile(jnk).parse(jnk.sheet_names[0])
            except:
                raise IOError('could not load excel file. Please convert to csv')
        else:
            try:
                df = pandas.read_table(spreadsheet)
            except:
                raise IOError('could not read spreadsheet. Try loading the df yourself with pandas and inputting that')
    else:
        df = spreadsheet
    
    print('loading beta images')
    if intercept == 'first':
        int_img = ni.load(beta_paths[0]).get_data()
        beta_paths.remove(beta_paths[0])
    elif intercept == 'last':
        int_img = ni.load(beta_paths[-1]).get_data()
        beta_paths.remove(beta_paths[-1])
    jnk = ni.concat_images(beta_paths)
    beta_imgs = jnk.get_data()
    aff = jnk.affine
    if type(image_mask) != type(None):
        try:
            mask = ni.load(image_mask).get_data()
        except:
            raise IOError('could not load mask. Please ensure the path points to an existing nifti image')
    
    
    def transform

    for i,scan in enumerate(scans):
        if i not in checkdf.index:
            continue
        sid = idf.loc[i,'subject']
        vis = idf.loc[i,'visit']
        ginf = idf.index[i]
        subvec = tdf[(tdf.ID == sid) & (tdf.visit == vis)]
        if len(subvec) > 1:
            subvec = subvec.loc[subvec.index[0]]
        agem = np.full_like(rage,subvec['AGE'])
        sexm = np.full_like(rsex,subvec['PTGENDER'])
        edum = np.full_like(redu,subvec['PTEDUCAT'])
        pmap = rmean + (agem * rage) + (sexm * rsex) + (edum * redu)
        jnk = ni.load(scan)
        aff = jnk.affine
        dat = jnk.get_data()
        wscr_mat = (dat - pmap) / rstd
        wscr_mat[msk==0] = 0
        wscr_img = ni.Nifti1Image(wscr_mat,aff)
        flnm = '%s_v%s_smooth_AV1451_wscored_SUVR'%(sid,vis)
        wscr_img.to_filename(os.path.join(wscr_dir,flnm))
        print('finished %s of %s'%(i+1,len(scans)))



In [3]:
ni.concat_images?